In [1]:
from kalman_gst import *
from pygsti.modelpacks import smq1Q_XYZI as std

In [5]:
from error_generators import ErrorGen

In [6]:
egen = ErrorGen(1)
egen.vec

/home/jpmarceaux/PycharmProjects/AnalyticErrorAnalysis/venv/lib/python3.10/site-packages/pygsti/modelmembers/operations/lindbladerrorgen.py:1691: UserWarning: Using 'CPTP' as a Lindblad type is deprecated, and you should now use 'CPTPLND'
  _warnings.warn("Using 'CPTP' as a Lindblad type is deprecated, and you should now use 'CPTPLND'")


[{H}^{X}_{Gq0},
 {H}^{Y}_{Gq0},
 {H}^{Z}_{Gq0},
 {S}^{X}_{Gq0},
 {C}^{XxY}_{Gq0},
 {A}^{XxY}_{Gq0},
 {C}^{XxZ}_{Gq0},
 {A}^{XxZ}_{Gq0},
 {S}^{Y}_{Gq0},
 {C}^{YxZ}_{Gq0},
 {A}^{YxZ}_{Gq0},
 {S}^{Z}_{Gq0}]

In [10]:
Sx = egen.vec[3]
Sz = egen.vec[-1]
Sy = egen.vec[8]
Hx = egen.vec[0]


In [12]:
print(Sx, Sy, Sz)

{S}^{X}_{Gq0} {S}^{Y}_{Gq0} {S}^{Z}_{Gq0}


In [13]:
egen.feature_of_param(Sx)+egen.feature_of_param(Sy)+egen.feature_of_param(Sz)

array([[ 0.,  0.,  0.,  0.],
       [ 0., -4.,  0.,  0.],
       [ 0.,  0., -4.,  0.],
       [ 0.,  0.,  0., -4.]])

In [2]:
shots_per_circuit = 256

# setup the FOGI model
mdl_datagen = std.target_model('GLND')
basis1q = pygsti.baseobjs.Basis.cast('pp', 4)
gauge_basis = pygsti.baseobjs.CompleteElementaryErrorgenBasis(
                        basis1q, mdl_datagen.state_space, elementary_errorgen_types='HS')
mdl_datagen.setup_fogi(gauge_basis, None, None, reparameterize=True,
                     dependent_fogi_action='drop', include_spam=True)
target_model = mdl_datagen.copy()

In [3]:
# add noise to the model
SEED = 3122
np.random.seed(SEED)

max_error_rate = 0.001
ar = mdl_datagen.fogi_errorgen_components_array(include_fogv=False, normalized_elem_gens=True)
ar = max_error_rate*np.random.rand(len(ar))
mdl_datagen.set_fogi_errorgen_components_array(ar, include_fogv=False, normalized_elem_gens=True)

mdl_datagen.from_vector(project_params_to_cp(mdl_datagen))

print('MSE with target', mserror(mdl_datagen, target_model))
print('agi with target', avg_gs_infidelity(mdl_datagen, target_model))
print('model is cptp', model_is_cptp(mdl_datagen))


KeyboardInterrupt



In [ ]:
class ExtendedKalmanFilter():
    """
    An extended Kalman filter for gate-set tomography

    --- Parameters ---
    model: an underlying pygsti model
    num_params: number of parameters in the pygsti model
    P: current covariance matrix
    """
    def __init__(self, model, P0):
        self.model = model.copy()
        self.P = P0

        self.param_history = [self.model.to_vector()]
        self.covar_history = [self.P]

    def update(self, circ, count_vec, clip_range=[-1,1], Q=None, R_additional=None, max_itr=1, itr_eps=1e-4):
        """
        Makes an exact update to the model
        where the jacobian is calculated as the current estimate

        --- Arguments ---
        circ: pygsti circuit used in the update
        count_vec: vector of observed counts
        clip_range: reasonable clipping range for the parameter update
        Q: state-space covariance
        R_additional: additional measurement covariance
        max_itr: maximum number of iterations to the update
        itr_eps: epsilon for minimum difference to end iterated updates

        --- Returns ---
        innovation: the prior innovation
        kgain: the Kalman gain
        """
        prior_covar = self.P
        prior_state = self.model.to_vector()
        hilbert_dims = 2**(circ.width)

        for itr in range(max_itr):
            # find the predicted frequency for the circuit outcome under the model
            probs = self.model.probabilities(circ)
            p_model = vector_from_outcomes(probs, hilbert_dims)

            # calculate the observed frequency
            total_counts = sum(count_vec)
            observation = count_vec/total_counts

            # calculate jacobian
            jacob = matrix_from_jacob(self.model.sim.dprobs(circ), 2**circ.width)

            # calculate the covaraiance of the observation
            mean_frequency = ( count_vec+np.ones(len(count_vec)) )/( sum(count_vec)+len(count_vec) )
            R = (1/(sum(count_vec)+len(count_vec)+1))*categorical_covar(mean_frequency)

            # add any additional noise
            if R_additional is not None:
                R += R_additional
            if Q is None:
                Q = 0*np.eye(self.model.num_params)

            # Kalman gain
            P = prior_covar + Q
            kgain = P@jacob.T@np.linalg.pinv(jacob@P@jacob.T + R, 1e-15)

            # Kalman update
            innovation = observation - p_model
            post_state = prior_state + kgain@innovation
            post_state = np.clip(post_state, clip_range[0], clip_range[1])

            # check if iteration should end
            if np.linalg.norm(post_state - prior_state) < itr_eps:
                break
            else:
                prior_state = post_state
                self.model.from_vector(post_state)

        # update class parameters
        self.P = (np.eye(self.model.num_params) - kgain@jacob)@P
        self.model.from_vector(post_state)

        self.param_history.append(post_state)
        self.covar_history.append(self.P)

        return innovation, kgain

In [28]:
germs = std.germs()
pfids = std.prep_fiducials()
mfids = std.meas_fiducials()

In [29]:
mse = mserror(mdl_datagen, target_model)

In [31]:
np.sqrt(mse)

0.00355919143352553

In [ ]:
max_circ_length =